In [2]:
import pandas as pd

In [115]:
# Make team ID and performance score columns

from data_helper_functions import get_JSON_dict

csv_filepath = "2023_vectors/features.csv"    

df = pd.read_csv(csv_filepath, index_col=[0])

metadata_columns = ["RESULT", "CHAMPION", "participantID", "SUMMONER_NAME", "ESPORTS_ID", "PLATFORM_ID"]

metadata = df[metadata_columns]
features = df.drop(columns=metadata_columns, axis=1)

esports_ids = metadata.ESPORTS_ID.unique()

team_meta_data = metadata[metadata["ESPORTS_ID"] == id]
all_data = df[df["ESPORTS_ID"] == id]


# update rows
team_names = []
performance_scores = []

importances = get_JSON_dict("./weights.json")
importances_keys = list(importances.keys())

teams = get_JSON_dict("data/esports-data/teams.json")
players = get_JSON_dict("data/esports-data/players.json")


for index, row in df.iterrows():
    power_score = 0

    # Calculate Power Score
    for key in importances_keys:
        power_score += (row[key] * importances[key])

    performance_scores.append(power_score)

    # Append Name
    handle = row["SUMMONER_NAME"]
    if handle.count(" ") > 0:
        index = handle.find(" ")
        handle = handle[(index + 1):]

    # Append team
    team_found = False
    for player in players:
        if handle == player["handle"]:
            team_names.append(player["home_team_id"])
            team_found = True
            break

    if team_found == False:
        team_names.append(None)


In [116]:
new_dataframe = df

In [117]:
len(performance_scores)
len(team_names)

73290

In [118]:
team_names.count(None)

5716

In [119]:
new_dataframe["POWER_SCORE"] = performance_scores
new_dataframe["TEAM_ID"] = team_names
new_dataframe

,level,shutdownValue,totalGold,MINIONS_KILLED,NEUTRAL_MINIONS_KILLED,NEUTRAL_MINIONS_KILLED_YOUR_JUNGLE,NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE,CHAMPIONS_KILLED,NUM_DEATHS,ASSISTS,...,TOTAL_HEAL_ON_TEAMMATES,participantID,CHAMPION,SUMMONER_NAME,ESPORTS_ID,PLATFORM_ID,SIDE,RESULT,POWER_SCORE,TEAM_ID
0,15,150,10450,178,6.000000,0.000000,6.000000,2,0,10,...,0.000000,1,Poppy,MIR Allorim,110733838936446929,ESPORTSTMNT03:3196037,100,1,6.146082,109981650516317055
1,14,400,11406,17,114.400017,67.500008,22.399998,5,2,10,...,0.000000,2,Rengar,MIR Chad,110733838936446929,ESPORTSTMNT03:3196037,100,1,10.010893,109981650516317055
2,14,300,10481,191,0.000000,0.000000,0.000000,4,4,8,...,0.000000,3,Kaisa,MIR Sudzzi,110733838936446929,ESPORTSTMNT03:3196037,100,1,5.684131,109981650516317055
3,13,700,13612,204,43.600006,19.999996,10.100000,10,1,6,...,0.000000,4,Xayah,MIR Neo,110733838936446929,ESPORTSTMNT03:3196037,100,1,13.501542,109981650516317055
4,13,0,7706,31,0.000000,0.000000,0.000000,1,0,16,...,1439.260132,5,Zilean,MIR Dardoch,110733838936446929,ESPORTSTMNT03:3196037,100,1,6.784700,109981650516317055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,16,300,11631,213,0.000000,0.000000,0.000000,3,0,10,...,1249.557007,1,Illaoi,GK Satorius,109545777184190004,ESPORTSTMNT01:3337315,100,1,6.846508,109696126554266494
696,14,150,10146,10,119.150024,73.400017,8.500000,2,0,14,...,1845.673462,2,Maokai,GK Viking,109545777184190004,ESPORTSTMNT01:3337315,100,1,9.084965,109696126554266494
697,16,150,12621,213,8.750000,7.000000,0.000000,5,4,12,...,0.000000,3,Cassiopeia,GK Giyuu,109545777184190004,ESPORTSTMNT01:3337315,100,1,4.771465,109696126554266494
698,15,700,15902,277,12.000000,8.000000,0.000000,10,2,7,...,760.497253,4,Varus,GK Rin,109545777184190004,ESPORTSTMNT01:3337315,100,1,13.355645,109696126554266494


In [120]:
esports_ids = new_dataframe.ESPORTS_ID.unique()

In [121]:
# If a player doesn't have a home team, try and assume that they are playing as a sub or guest on a team for a given game
# If there is more than 1 guest player in a game, delete the game, having guests can skew the performance of the team

games_deleted_count = 0

for id in esports_ids:
    blue = new_dataframe[(new_dataframe["ESPORTS_ID"] == id) & (new_dataframe["SIDE"] == 100)]
    red = new_dataframe[(new_dataframe["ESPORTS_ID"] == id) & (new_dataframe["SIDE"] == 200)]


    blue_nones = blue[blue["TEAM_ID"] == None]
    red_nones = red[red["TEAM_ID"] == None]

    if len(blue_nones) > 0:
        if len(blue_nones) == 1:
            index = blue_nones.index
            blue_non_none = blue[blue["TEAM_ID"] != None]
            new_dataframe.at[index,"TEAM_ID"] = blue_non_none.TEAM_ID.unique()[0]
        else:
            games_deleted_count += 1
            new_dataframe.drop(blue.index)

    if len(red_nones) > 0:
        if len(red_nones) == 1:
            index = red_nones.index
            red_non_none = red[red["TEAM_ID"] != None]
            new_dataframe.at[index,"TEAM_ID"] = red_non_none.TEAM_ID.unique()[0]
        else:
            games_deleted_count += 1
            new_dataframe.drop(red.index)


print(games_deleted_count/2)
    

0.0


In [122]:
new_dataframe

,level,shutdownValue,totalGold,MINIONS_KILLED,NEUTRAL_MINIONS_KILLED,NEUTRAL_MINIONS_KILLED_YOUR_JUNGLE,NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE,CHAMPIONS_KILLED,NUM_DEATHS,ASSISTS,...,TOTAL_HEAL_ON_TEAMMATES,participantID,CHAMPION,SUMMONER_NAME,ESPORTS_ID,PLATFORM_ID,SIDE,RESULT,POWER_SCORE,TEAM_ID
0,15,150,10450,178,6.000000,0.000000,6.000000,2,0,10,...,0.000000,1,Poppy,MIR Allorim,110733838936446929,ESPORTSTMNT03:3196037,100,1,6.146082,109981650516317055
1,14,400,11406,17,114.400017,67.500008,22.399998,5,2,10,...,0.000000,2,Rengar,MIR Chad,110733838936446929,ESPORTSTMNT03:3196037,100,1,10.010893,109981650516317055
2,14,300,10481,191,0.000000,0.000000,0.000000,4,4,8,...,0.000000,3,Kaisa,MIR Sudzzi,110733838936446929,ESPORTSTMNT03:3196037,100,1,5.684131,109981650516317055
3,13,700,13612,204,43.600006,19.999996,10.100000,10,1,6,...,0.000000,4,Xayah,MIR Neo,110733838936446929,ESPORTSTMNT03:3196037,100,1,13.501542,109981650516317055
4,13,0,7706,31,0.000000,0.000000,0.000000,1,0,16,...,1439.260132,5,Zilean,MIR Dardoch,110733838936446929,ESPORTSTMNT03:3196037,100,1,6.784700,109981650516317055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,16,300,11631,213,0.000000,0.000000,0.000000,3,0,10,...,1249.557007,1,Illaoi,GK Satorius,109545777184190004,ESPORTSTMNT01:3337315,100,1,6.846508,109696126554266494
696,14,150,10146,10,119.150024,73.400017,8.500000,2,0,14,...,1845.673462,2,Maokai,GK Viking,109545777184190004,ESPORTSTMNT01:3337315,100,1,9.084965,109696126554266494
697,16,150,12621,213,8.750000,7.000000,0.000000,5,4,12,...,0.000000,3,Cassiopeia,GK Giyuu,109545777184190004,ESPORTSTMNT01:3337315,100,1,4.771465,109696126554266494
698,15,700,15902,277,12.000000,8.000000,0.000000,10,2,7,...,760.497253,4,Varus,GK Rin,109545777184190004,ESPORTSTMNT01:3337315,100,1,13.355645,109696126554266494


In [158]:
# Finally add team names
teams = get_JSON_dict("data/esports-data/teams.json")
team_names = []

for index, row in new_dataframe.iterrows():
    found = False
    for team in teams:
        if str(row["TEAM_ID"]) == team["team_id"]:
            found = True
            team_names.append(team["name"])
            break
    if found == False:
        team_names.append("NA")

new_dataframe["TEAM_NAME"] = team_names

In [159]:
new_dataframe.head(30)

,level,shutdownValue,totalGold,MINIONS_KILLED,NEUTRAL_MINIONS_KILLED,NEUTRAL_MINIONS_KILLED_YOUR_JUNGLE,NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE,CHAMPIONS_KILLED,NUM_DEATHS,ASSISTS,...,participantID,CHAMPION,SUMMONER_NAME,ESPORTS_ID,PLATFORM_ID,SIDE,RESULT,POWER_SCORE,TEAM_ID,TEAM_NAME
0,15,150,10450,178,6.000000,0.000000,6.000000,2,0,10,...,1,Poppy,MIR Allorim,110733838936446929,ESPORTSTMNT03:3196037,100,1,6.146082,109981650516317055,Mirage Alliance
1,14,400,11406,17,114.400017,67.500008,22.399998,5,2,10,...,2,Rengar,MIR Chad,110733838936446929,ESPORTSTMNT03:3196037,100,1,10.010893,109981650516317055,Mirage Alliance
2,14,300,10481,191,0.000000,0.000000,0.000000,4,4,8,...,3,Kaisa,MIR Sudzzi,110733838936446929,ESPORTSTMNT03:3196037,100,1,5.684131,109981650516317055,Mirage Alliance
3,13,700,13612,204,43.600006,19.999996,10.100000,10,1,6,...,4,Xayah,MIR Neo,110733838936446929,ESPORTSTMNT03:3196037,100,1,13.501542,109981650516317055,Mirage Alliance
4,13,0,7706,31,0.000000,0.000000,0.000000,1,0,16,...,5,Zilean,MIR Dardoch,110733838936446929,ESPORTSTMNT03:3196037,100,1,6.784700,109981650516317055,Mirage Alliance
5,14,0,8978,190,0.000000,0.000000,0.000000,1,6,0,...,6,Darius,ROTM Kookykrook,110733838936446929,ESPORTSTMNT03:3196037,200,0,-2.560551,110733889090164854,Return of the Middlesticks
6,11,0,7002,4,105.000015,77.000015,12.000001,1,1,3,...,7,Ivern,ROTM Dragowski,110733838936446929,ESPORTSTMNT03:3196037,200,0,-1.787484,110733889090164854,Return of the Middlesticks
7,14,0,9444,227,0.000000,0.000000,0.000000,3,4,1,...,8,Leblanc,ROTM Max108,110733838936446929,ESPORTSTMNT03:3196037,200,0,-4.078831,110733889090164854,Return of the Middlesticks
8,12,0,10108,197,8.000000,8.000000,0.000000,2,4,0,...,9,Aphelios,ROTM Puppeh,110733838936446929,ESPORTSTMNT03:3196037,200,0,-3.178553,110733889090164854,Return of the Middlesticks
9,10,0,5820,41,0.000000,0.000000,0.000000,0,7,4,...,10,Rell,ROTM Boy Wonder,110733838936446929,ESPORTSTMNT03:3196037,200,0,-1.235588,110733889090164854,Return of the Middlesticks


In [160]:
# Save to new csv
new_dataframe.to_csv("2023_vectors/all_features.csv")

In [163]:
test = pd.read_csv("2023_vectors/all_features.csv", index_col=[0])

In [166]:
test.head(30)

,level,shutdownValue,totalGold,MINIONS_KILLED,NEUTRAL_MINIONS_KILLED,NEUTRAL_MINIONS_KILLED_YOUR_JUNGLE,NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE,CHAMPIONS_KILLED,NUM_DEATHS,ASSISTS,...,participantID,CHAMPION,SUMMONER_NAME,ESPORTS_ID,PLATFORM_ID,SIDE,RESULT,POWER_SCORE,TEAM_ID,TEAM_NAME
0,15,150,10450,178,6.000000,0.000000,6.000000,2,0,10,...,1,Poppy,MIR Allorim,110733838936446929,ESPORTSTMNT03:3196037,100,1,6.146082,1.099817e+17,Mirage Alliance
1,14,400,11406,17,114.400017,67.500008,22.399998,5,2,10,...,2,Rengar,MIR Chad,110733838936446929,ESPORTSTMNT03:3196037,100,1,10.010893,1.099817e+17,Mirage Alliance
2,14,300,10481,191,0.000000,0.000000,0.000000,4,4,8,...,3,Kaisa,MIR Sudzzi,110733838936446929,ESPORTSTMNT03:3196037,100,1,5.684131,1.099817e+17,Mirage Alliance
3,13,700,13612,204,43.600006,19.999996,10.100000,10,1,6,...,4,Xayah,MIR Neo,110733838936446929,ESPORTSTMNT03:3196037,100,1,13.501542,1.099817e+17,Mirage Alliance
4,13,0,7706,31,0.000000,0.000000,0.000000,1,0,16,...,5,Zilean,MIR Dardoch,110733838936446929,ESPORTSTMNT03:3196037,100,1,6.784700,1.099817e+17,Mirage Alliance
5,14,0,8978,190,0.000000,0.000000,0.000000,1,6,0,...,6,Darius,ROTM Kookykrook,110733838936446929,ESPORTSTMNT03:3196037,200,0,-2.560551,1.107339e+17,Return of the Middlesticks
6,11,0,7002,4,105.000015,77.000015,12.000001,1,1,3,...,7,Ivern,ROTM Dragowski,110733838936446929,ESPORTSTMNT03:3196037,200,0,-1.787484,1.107339e+17,Return of the Middlesticks
7,14,0,9444,227,0.000000,0.000000,0.000000,3,4,1,...,8,Leblanc,ROTM Max108,110733838936446929,ESPORTSTMNT03:3196037,200,0,-4.078831,1.107339e+17,Return of the Middlesticks
8,12,0,10108,197,8.000000,8.000000,0.000000,2,4,0,...,9,Aphelios,ROTM Puppeh,110733838936446929,ESPORTSTMNT03:3196037,200,0,-3.178553,1.107339e+17,Return of the Middlesticks
9,10,0,5820,41,0.000000,0.000000,0.000000,0,7,4,...,10,Rell,ROTM Boy Wonder,110733838936446929,ESPORTSTMNT03:3196037,200,0,-1.235588,1.107339e+17,Return of the Middlesticks
